In [13]:
import dask
import os
import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import importlib

from letkf_forecasting import analyse_results, letkf_io

In [14]:
importlib.reload(analyse_results)
importlib.reload(letkf_io)

base_folder = '/a2/uaren/travis'

file_name = 'all_days'

decimals = 3

runs = [ 'owp_opt', 'persistence', 'opt_flow', 'wrf_no_div',
        'opt_flow_with_div', 'wrf', 'radiosonde', 'wrf_mean']
# runs = ['wrf_mean']

In [15]:
save_directory = "/home2/travis/python_code/letkf_forecasting/tables/"

In [16]:
horizons = [15, 30, 45, 60]

In [17]:
legend_dict = {'opt_flow': 'Opt. Flow', 
               'opt_flow_with_div': 'Opt. Flow w/ Div.',
               'wrf_no_div': 'WRF',
               'wrf': 'WRF w/ Div.',
               'owp_opt': 'BACON',                                                                                
               'persistence': 'Persis.',
               'radiosonde': 'Radiosonde',
               'wrf_mean': 'WRF Mean',
               'ens_member': 'Ens. Member'} 

In [18]:
rmse = pd.DataFrame(index=horizons, columns=runs)
rmse.index.name = 'Horizon'
correlation = rmse.copy()
bias = rmse.copy()
for run_name in runs:
    results_folder_path = os.path.join(                                          
        base_folder,                                                               
        'results',                                                               
        'multi_day_error',
        'third_set',                                                             
        run_name)
    stat_name = 'rmse'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    rmse[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'bias'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    bias[run_name] = pd.read_hdf(file_path, stat_name)
    
    stat_name = 'correlation'
    file_path = os.path.join(results_folder_path,
                             f'{stat_name}.h5')
    correlation[run_name] = pd.read_hdf(file_path, stat_name)

In [19]:
peices = [rmse, correlation, bias]
combined = pd.concat(peices, axis=0,
                     keys=['RMSE', 'Corr.', 'Bias'])
combined = combined.rename(columns=legend_dict)

In [20]:
def is_empty(str):
    return str != ''

In [25]:
def format_table(text, header_num=5, footer_num=2):
    text = text.split(' ')
    text = list(filter(is_empty, text))
    text = ' '.join(text)
    split_text = text.split('\n')
    split_titles2 = split_text[2].split('&')
    split_titles3 = split_text[3].split('&')
    for num, title in enumerate(split_text[2].split('&')):
        if ' w/ Div.' in title:
            split_titles2[num] = title[:-8]
            split_titles3[num] = title[-9:]
    split_titles2 = '&'.join(split_titles2)
    split_titles3 = '&'.join(split_titles3)
    split_text[2] = split_titles2
    split_text[3] = split_titles3
    
    for line_num, line in enumerate(split_text[header_num:-footer_num - 1]):
        split_line = line.split(' ')
        if split_line[0] == 'Corr.':
            Corr = True
        elif split_line[0] != '':
            Corr = False
        num_slice = slice(4, None, 2)
        numbers_str = split_line[num_slice]
        numbers = np.array(
            split_line[num_slice],
            dtype='float')
        if Corr:
            best_num = numbers.max()
        else:
            best_num = numbers[np.abs(numbers).argmin()]
        argmins = np.where(numbers == best_num)[0]
#         numbers = list(numbers.astype('str'))
        for argmin in argmins:
            numbers_str[argmin] = '\\textbf{' + numbers_str[argmin] + '}'
        split_line[num_slice] = numbers_str
        split_text[header_num + line_num] = ' '.join(split_line)
    
#     for count in range(hor_num - 1):
#         split_line.insert(((count + 1)*run_num)*2 + 2 + count, '&')
#     split_line

    
    return '\n'.join(split_text)

In [29]:
column_format = 'll' + 'c' * len(runs)
text = combined.round(decimals=decimals).to_latex(column_format=column_format)
text2 = format_table(text)
print(text2)

\begin{tabular}{llcccccccc}
\toprule
 & & BACON & Persis. & Opt. Flow & WRF & Opt. Flow & WRF & Radiosonde & WRF Mean \\
{} & Horizon & & & & & w/ Div. & w/ Div. & & \\
\midrule
RMSE & 15 & \textbf{0.108} & 0.124 & 0.113 & 0.118 & 0.131 & 0.118 & 0.139 & 0.116 \\
 & 30 & \textbf{0.124} & 0.143 & 0.133 & 0.142 & 0.162 & 0.144 & 0.154 & 0.140 \\
 & 45 & \textbf{0.127} & 0.154 & 0.145 & 0.147 & 0.180 & 0.151 & 0.159 & 0.149 \\
 & 60 & \textbf{0.132} & 0.158 & 0.153 & 0.150 & 0.190 & 0.155 & 0.164 & 0.152 \\
Corr. & 15 & \textbf{0.877} & 0.844 & 0.869 & 0.855 & 0.819 & 0.853 & 0.797 & 0.858 \\
 & 30 & \textbf{0.837} & 0.791 & 0.816 & 0.786 & 0.716 & 0.779 & 0.747 & 0.790 \\
 & 45 & \textbf{0.826} & 0.759 & 0.778 & 0.770 & 0.652 & 0.756 & 0.728 & 0.763 \\
 & 60 & \textbf{0.808} & 0.743 & 0.753 & 0.759 & 0.611 & 0.740 & 0.708 & 0.750 \\
Bias & 15 & \textbf{-0.001} & -0.002 & \textbf{-0.001} & -0.007 & -0.004 & -0.008 & -0.006 & -0.006 \\
 & 30 & \textbf{-0.005} & \textbf{-0.005} & \textbf{-0

In [30]:
this_file = os.path.join(save_directory, f'{file_name}_results.tex')
with open(this_file, 'w') as file:
    file.write(text2)